<a href="https://colab.research.google.com/github/Kirt10/ProspectorAI/blob/main/ProspectorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import PyPDF2
from transformers import pipeline
import re

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def preprocess_text(text):
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

def analyze_text_for_investor(text):
    keywords = {
        "future_growth": ["growth", "expansion", "increase", "opportunities", "forecast", "outlook"],
        "key_changes": ["change", "restructuring", "new strategy", "acquisition", "partnership"],
        "key_triggers": ["trigger", "catalyst", "event", "impact", "driver"],
        "material_info": ["earnings", "revenue", "profit", "loss", "guidance", "forecast"]
    }

    nlp = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

    sentences = re.split(r'(?<=[.!?]) +', text)

    results = []
    for sentence in sentences:
        for category, keys in keywords.items():
            if any(key in sentence.lower() for key in keys):
                sentiment = nlp(sentence)[0]
                if sentiment['label'] == 'POSITIVE' and sentiment['score'] > 0.8:
                    results.append({
                        "category": category,
                        "sentence": sentence,
                        "confidence": sentiment['score']
                    })
    return results

def summarize_investment_info(relevant_info):
    summary = {
        "future_growth": [],
        "key_changes": [],
        "key_triggers": [],
        "material_info": []
    }

    for info in relevant_info:
        category = info["category"]
        summary[category].append(f"- {info['sentence']} (Confidence: {info['confidence']:.2f})")

    output = "Key Investment Insights:\n"
    for category, points in summary.items():
        if points:
            output += f"\n{category.replace('_', ' ').title()}:\n"
            output += "\n".join(points) + "\n"
    return output

def main(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    text = preprocess_text(text)
    relevant_info = analyze_text_for_investor(text)

    summary = summarize_investment_info(relevant_info)
    print(summary)


if __name__ == "__main__":
    pdf_path = '/content/SJS Transcript Call.pdf'  #"enter your pdf path"
    main(pdf_path)

Device set to use cpu


Key Investment Insights:

Future Growth:
- 
Sanjay will take you all through the industry view , our business performance and also give a 
strategic outloo k for the future growth of the company at the end. (Confidence: 1.00)
- 
After the  successful acquisition of Exot ech and the robust performance we delivered there in the 
last two years, we have gained more confidence in our execution capabilities of acquiring and 
integrating com panies that could take SJS to the next  level of growth. (Confidence: 1.00)
- 
Walter Pack  acquisition has ope ned up a plethora of new opportunities for us. (Confidence: 1.00)
- 
With Walter Pack acquisition, we have not only acquired a very large growth and high margin 
business at a good valuation, but at the same time achieved many k ey strategic objectives in 
terms of addition of new and emerging technologies, new customers, manufacturing capabilities 
and increase management bandwidth. (Confidence: 1.00)
- I am excited and 
eagerly looking forwar